In [19]:
import os
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

# Настройка графики
sns.set(style="whitegrid")
plt.rcParams.update({'figure.figsize': (10, 6)})

# Загрузка переменных окружения
load_dotenv()
API_URL = os.getenv("API_URL")
DATE_BEGIN = os.getenv("DATE_BEGIN")
DATE_END = os.getenv("DATE_END")

# ==== 1. Загрузка данных по API ====
visits_url = f"{API_URL}/visits?begin={DATE_BEGIN}&end={DATE_END}"
regs_url = f"{API_URL}/registrations?begin={DATE_BEGIN}&end={DATE_END}"

visits_df = pd.DataFrame(requests.get(visits_url).json())
regs_df = pd.DataFrame(requests.get(regs_url).json())

# ==== 2. Очистка и подготовка ====
visits_df["date"] = pd.to_datetime(visits_df["datetime"]).dt.date
regs_df["date"] = pd.to_datetime(regs_df["datetime"]).dt.date

# Удаляем ботов
visits_df = visits_df[~visits_df["user_agent"].str.contains("bot", case=False, na=False)]

# Берем только последний визит для каждого visit_id
visits_df = visits_df.sort_values("datetime").drop_duplicates("visit_id", keep="last")

# ==== 3. Агрегация и расчет конверсии ====
visits_grouped = visits_df.groupby(["date", "platform"]).size().reset_index(name="visits")
regs_grouped = regs_df.groupby(["date", "platform"]).size().reset_index(name="registrations")

merged_df = pd.merge(visits_grouped, regs_grouped, on=["date", "platform"], how="outer")
merged_df["visits"] = merged_df["visits"].fillna(0).astype(int)
merged_df["registrations"] = merged_df["registrations"].fillna(0).astype(int)

merged_df["conversion"] = merged_df.apply(
    lambda row: (row["registrations"] / row["visits"] * 100) if row["visits"] > 0 else 0, axis=1
)
merged_df = merged_df.sort_values("date")

# ==== 4. Сохраняем conversion.json ====
conversion_df = merged_df.rename(columns={"date": "date_group"})[["date_group", "platform", "visits", "registrations", "conversion"]]
conversion_df.to_json("conversion.json", orient="columns")

# ==== 5. Загрузка и сохранение рекламы ====
ads_id = "12vCtGhJlcK_CBcs8ES3BfEPbk6OJ45Qj"
ads_url = f"https://drive.google.com/uc?export=download&id={ads_id}"
ads_response = requests.get(ads_url, timeout=10)

os.makedirs("backend/data_generator/data", exist_ok=True)
with open("ads.csv", "wb") as f:
    f.write(ads_response.content)

ads_df = pd.read_csv("ads.csv")
ads_df["date"] = pd.to_datetime(ads_df["date"]).dt.date
ads_df["cost"] = pd.to_numeric(ads_df["cost"], errors="coerce").fillna(0)
ads_grouped = ads_df.groupby(["date", "utm_campaign"]).agg(cost=("cost", "sum")).reset_index()

# ==== 6. Сохраняем ads.json ====
ads_merged = pd.merge(merged_df, ads_grouped, on="date", how="left").fillna(0)
ads_merged.to_json("ads.json", orient="columns")

# ==== 7. Построение графиков ====
os.makedirs("charts", exist_ok=True)

# Итоговые визиты
total_visits = merged_df.groupby("date")["visits"].sum()
total_visits.plot(title="Итоговые визиты")
plt.xlabel("Дата")
plt.ylabel("Визиты")
plt.tight_layout()
plt.savefig("charts/total_visits.png")
plt.clf()

# Визиты по платформам
sns.lineplot(data=merged_df, x="date", y="visits", hue="platform")
plt.title("Визиты по платформам")
plt.xlabel("Дата")
plt.ylabel("Визиты")
plt.tight_layout()
plt.savefig("charts/visits_by_platform.png")
plt.clf()

# Итоговые регистрации
total_regs = merged_df.groupby("date")["registrations"].sum()
total_regs.plot(title="Итоговые регистрации")
plt.xlabel("Дата")
plt.ylabel("Регистрации")
plt.tight_layout()
plt.savefig("charts/total_registrations.png")
plt.clf()

# Регистрации по платформам
sns.lineplot(data=merged_df, x="date", y="registrations", hue="platform")
plt.title("Регистрации по платформам")
plt.xlabel("Дата")
plt.ylabel("Регистрации")
plt.tight_layout()
plt.savefig("charts/registrations_by_platform.png")
plt.clf()

# Конверсия по платформам
sns.lineplot(data=merged_df, x="date", y="conversion", hue="platform")
plt.title("Конверсия по платформам")
plt.xlabel("Дата")
plt.ylabel("Конверсия (%)")
plt.tight_layout()
plt.savefig("charts/conversion_by_platform.png")
plt.clf()

# Средняя конверсия
avg_conv = merged_df.groupby("platform")["conversion"].mean().reset_index()
sns.barplot(data=avg_conv, x="platform", y="conversion")
plt.title("Средняя конверсия по платформам")
plt.ylabel("Конверсия (%)")
plt.tight_layout()
plt.savefig("charts/avg_conversion_by_platform.png")
plt.clf()

# Стоимость рекламы по датам
ads_costs = ads_grouped.groupby("date")["cost"].sum().reset_index()
sns.lineplot(data=ads_costs, x="date", y="cost")
plt.title("Стоимость рекламы")
plt.xlabel("Дата")
plt.ylabel("Стоимость")
plt.tight_layout()
plt.savefig("charts/ads_costs.png")
plt.clf()

# Визиты с цветом utm_campaign
merged_ads_visits = pd.merge(visits_grouped, ads_grouped, on="date", how="left")
sns.scatterplot(data=merged_ads_visits, x="date", y="visits", hue="utm_campaign", palette="tab10")
plt.title("Визиты с цветовым кодом по UTM-кампании")
plt.tight_layout()
plt.savefig("charts/visits_by_campaign.png")
plt.clf()

# Регистрации с цветом utm_campaign
merged_ads_regs = pd.merge(regs_grouped, ads_grouped, on="date", how="left")
sns.scatterplot(data=merged_ads_regs, x="date", y="registrations", hue="utm_campaign", palette="tab10")
plt.title("Регистрации с цветовым кодом по UTM-кампании")
plt.tight_layout()
plt.savefig("charts/registrations_by_campaign.png")
plt.clf()

print("✅ Все файлы успешно созданы:")
print(" - conversion.json")
print(" - ads.json")
print(" - ads.csv")
print(" - 9 PNG-графиков в ./charts")


C:\Users\Nadezhda\AppData\Local\Temp\ipykernel_4344\250745656.py:88: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
C:\Users\Nadezhda\AppData\Local\Temp\ipykernel_4344\250745656.py:97: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
C:\Users\Nadezhda\AppData\Local\Temp\ipykernel_4344\250745656.py:106: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
C:\Users\Nadezhda\AppData\Local\Temp\ipykernel_4344\250745656.py:115: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
C:\Users\Nadezhda\AppData\Local\Temp\ipykernel_4344\250745656.py:124: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
C:\Users\Nadezhda\AppData\Local\Temp\ipykernel_4344\250745656.py:134: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
C:\Users\Nadezhda\AppData\Local\Temp\ipykernel_4344\250745656.py:142: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
C

✅ Все файлы успешно созданы:
 - conversion.json
 - ads.json
 - ads.csv
 - 9 PNG-графиков в ./charts
